In [ ]:
# | default_exp srtfile

In [ ]:
# | export
import re
import os
import sys
import codecs
from collections import UserList
from itertools import chain
from copy import copy
from datetime import time

In [ ]:
# | export
# === Missing helper classes ===
class InvalidItem(Exception):
    pass


class Error(Exception):
    pass


class InvalidTimeString(Exception):
    pass


class ComparableMixin:
    def __lt__(self, other):
        return self._cmpkey() < other._cmpkey()

    def __eq__(self, other):
        return self._cmpkey() == other._cmpkey()


class TimeItemDescriptor:
    def __init__(self, ratio, maxval=None):
        self.ratio = ratio
        self.maxval = maxval

    def __get__(self, instance, owner):
        if instance is None:
            return self
        val = instance.ordinal // self.ratio
        if self.maxval:
            val %= self.maxval // self.ratio
        return int(val)


# === SubRipTime ===
class SubRipTime(ComparableMixin):
    TIME_PATTERN = "%02d:%02d:%02d,%03d"
    TIME_REPR = "SubRipTime(%d, %d, %d, %d)"
    RE_TIME_SEP = re.compile(r"[:.,]")
    RE_INTEGER = re.compile(r"^(\d+)")
    SECONDS_RATIO = 1000
    MINUTES_RATIO = SECONDS_RATIO * 60
    HOURS_RATIO = MINUTES_RATIO * 60

    hours = TimeItemDescriptor(HOURS_RATIO)
    minutes = TimeItemDescriptor(MINUTES_RATIO, HOURS_RATIO)
    seconds = TimeItemDescriptor(SECONDS_RATIO, MINUTES_RATIO)
    milliseconds = TimeItemDescriptor(1, SECONDS_RATIO)

    def __init__(self, hours=0, minutes=0, seconds=0, milliseconds=0):
        self.ordinal = (
            hours * self.HOURS_RATIO
            + minutes * self.MINUTES_RATIO
            + seconds * self.SECONDS_RATIO
            + milliseconds
        )

    def __repr__(self):
        return self.TIME_REPR % tuple(self)

    def __str__(self):
        if self.ordinal < 0:
            return str(SubRipTime.from_ordinal(0))
        return self.TIME_PATTERN % tuple(self)

    def __iter__(self):
        yield self.hours
        yield self.minutes
        yield self.seconds
        yield self.milliseconds

    def _cmpkey(self):
        return self.ordinal

    def __add__(self, other):
        return self.from_ordinal(self.ordinal + self.coerce(other).ordinal)

    def __iadd__(self, other):
        self.ordinal += self.coerce(other).ordinal
        return self

    def __sub__(self, other):
        return self.from_ordinal(self.ordinal - self.coerce(other).ordinal)

    def __isub__(self, other):
        self.ordinal -= self.coerce(other).ordinal
        return self

    def __mul__(self, ratio):
        return self.from_ordinal(int(round(self.ordinal * ratio)))

    def __imul__(self, ratio):
        self.ordinal = int(round(self.ordinal * ratio))
        return self

    def shift(self, *args, **kwargs):
        if "ratio" in kwargs:
            self *= kwargs.pop("ratio")
        self += self.__class__(*args, **kwargs)

    @classmethod
    def from_ordinal(cls, ordinal):
        return cls(milliseconds=int(ordinal))

    @classmethod
    def from_string(cls, source):
        items = cls.RE_TIME_SEP.split(source)
        if len(items) != 4:
            raise InvalidTimeString(source)
        return cls(*(cls.parse_int(i) for i in items))

    @classmethod
    def from_time(cls, source):
        return cls(
            source.hour, source.minute, source.second, source.microsecond // 1000
        )

    def to_time(self):
        return time(self.hours, self.minutes, self.seconds, self.milliseconds * 1000)

    @classmethod
    def parse_int(cls, digits):
        try:
            return int(digits)
        except ValueError:
            match = cls.RE_INTEGER.match(digits)
            return int(match.group()) if match else 0

    @classmethod
    def coerce(cls, other):
        if isinstance(other, SubRipTime):
            return other
        if isinstance(other, str):
            return cls.from_string(other)
        if isinstance(other, int):
            return cls.from_ordinal(other)
        if isinstance(other, time):
            return cls.from_time(other)
        try:
            return cls(**other)
        except TypeError:
            return cls(*other)


# === SubRipItem ===
class SubRipItem(ComparableMixin):
    ITEM_PATTERN = "%s\n%s --> %s%s\n%s\n"
    TIMESTAMP_SEPARATOR = "-->"

    def __init__(self, index=0, start=None, end=None, text="", position=""):
        try:
            self.index = int(index)
        except (ValueError, TypeError):
            self.index = index
        self.start = SubRipTime.coerce(start or 0)
        self.end = SubRipTime.coerce(end or 0)
        self.text = str(text)
        self.position = str(position)

    def _cmpkey(self):
        return (self.start, self.end)

    def __str__(self):
        pos = f" {self.position}" if self.position.strip() else ""
        return self.ITEM_PATTERN % (self.index, self.start, self.end, pos, self.text)

    @property
    def duration(self):
        return self.end - self.start

    @property
    def text_without_tags(self):
        return re.sub(r"<[^>]*?>", "", self.text)

    @property
    def characters_per_second(self):
        chars = len(self.text_without_tags.replace("\n", ""))
        try:
            return chars / (self.duration.ordinal / 1000.0)
        except ZeroDivisionError:
            return 0.0

    def shift(self, *args, **kwargs):
        self.start.shift(*args, **kwargs)
        self.end.shift(*args, **kwargs)

    @classmethod
    def from_string(cls, source):
        return cls.from_lines(source.splitlines(True))

    @classmethod
    def from_lines(cls, lines):
        if len(lines) < 2:
            raise InvalidItem()
        lines = [line.rstrip() for line in lines]
        index = None
        if cls.TIMESTAMP_SEPARATOR not in lines[0]:
            index = lines.pop(0)
        start, end, pos = cls.split_timestamps(lines[0])
        return cls(index, start, end, "\n".join(lines[1:]), pos)

    @classmethod
    def split_timestamps(cls, line):
        parts = line.split(cls.TIMESTAMP_SEPARATOR)
        if len(parts) != 2:
            raise InvalidItem()
        start = parts[0].strip()
        end_pos = parts[1].lstrip().split(" ", 1)
        end = end_pos[0].strip()
        pos = end_pos[1].strip() if len(end_pos) > 1 else ""
        return (start, end, pos)


# === BOM definitions ===
BOMS = [
    (codecs.BOM_UTF32_LE, "utf_32_le"),
    (codecs.BOM_UTF32_BE, "utf_32_be"),
    (codecs.BOM_UTF16_LE, "utf_16_le"),
    (codecs.BOM_UTF16_BE, "utf_16_be"),
    (codecs.BOM_UTF8, "utf_8"),
]
CODECS_BOMS = {codec: bom.decode(codec) for bom, codec in BOMS}
BIGGER_BOM = max(len(bom) for bom, _ in BOMS)


# === SubRipFile ===
class SubRipFile(UserList):
    ERROR_PASS = 0
    ERROR_LOG = 1
    ERROR_RAISE = 2
    DEFAULT_ENCODING = "utf_8"

    def __init__(self, items=None, eol=None, path=None, encoding="utf-8"):
        super().__init__(items or [])
        self._eol = eol
        self.path = path
        self.encoding = encoding

    @property
    def eol(self):
        return self._eol or os.linesep

    @eol.setter
    def eol(self, value):
        self._eol = self._eol or value

    def slice(
        self, starts_before=None, starts_after=None, ends_before=None, ends_after=None
    ):
        clone = copy(self)
        if starts_before:
            clone.data = (i for i in clone.data if i.start < starts_before)
        if starts_after:
            clone.data = (i for i in clone.data if i.start > starts_after)
        if ends_before:
            clone.data = (i for i in clone.data if i.end < ends_before)
        if ends_after:
            clone.data = (i for i in clone.data if i.end > ends_after)
        clone.data = list(clone.data)
        return clone

    def at(self, timestamp=None, **kwargs):
        time = timestamp or kwargs
        return self.slice(starts_before=time, ends_after=time)

    def shift(self, *args, **kwargs):
        for item in self:
            item.shift(*args, **kwargs)

    def clean_indexes(self):
        self.sort()
        for idx, item in enumerate(self):
            item.index = idx + 1

    @property
    def text(self):
        return "\n".join(i.text for i in self)

    @classmethod
    def open(cls, path="", encoding=None, error_handling=ERROR_PASS):
        src_file, enc = cls._open_unicode_file(path, encoding)
        new_file = cls(path=path, encoding=enc)
        new_file.read(src_file, error_handling=error_handling)
        src_file.close()
        return new_file

    @classmethod
    def from_string(cls, source, **kwargs):
        handler = kwargs.pop("error_handling", None)
        inst = cls(**kwargs)
        inst.read(source.splitlines(True), error_handling=handler)
        return inst

    def read(self, source_file, error_handling=ERROR_PASS):
        self.eol = self._guess_eol(source_file)
        self.extend(self.stream(source_file, error_handling=error_handling))
        return self

    @classmethod
    def stream(cls, source_file, error_handling=ERROR_PASS):
        buffer = []
        for index, line in enumerate(chain(source_file, ["\n"])):
            if line.strip():
                buffer.append(line)
            else:
                source = buffer
                buffer = []
                if source and all(source):
                    try:
                        yield SubRipItem.from_lines(source)
                    except Error as e:
                        e.args += ("".join(source),)
                        cls._handle_error(e, error_handling, index)

    def save(self, path=None, encoding=None, eol=None):
        path = path or self.path
        encoding = encoding or self.encoding
        with codecs.open(path, "w+", encoding=encoding) as f:
            self.write_into(f, eol=eol)

    def write_into(self, f, eol=None):
        eol = eol or self.eol
        for item in self:
            s = str(item).replace("\n", eol) if eol != "\n" else str(item)
            f.write(s)
            if not s.endswith(2 * eol):
                f.write(eol)

    @classmethod
    def _guess_eol(cls, iterable):
        first = cls._get_first_line(iterable)
        for e in ("\r\n", "\r", "\n"):
            if first.endswith(e):
                return e
        return os.linesep

    @classmethod
    def _get_first_line(cls, iterable):
        if hasattr(iterable, "tell"):
            pos = iterable.tell()
        else:
            pos = None
        try:
            first = next(iter(iterable))
        except StopIteration:
            return ""
        if hasattr(iterable, "seek") and pos is not None:
            iterable.seek(pos)
        return first

    @classmethod
    def _detect_encoding(cls, path):
        with open(path, "rb") as f:
            first = f.read(BIGGER_BOM)
        for bom, encoding in BOMS:
            if first.startswith(bom):
                return encoding
        return cls.DEFAULT_ENCODING

    @classmethod
    def _open_unicode_file(cls, path, encoding=None):
        enc = encoding or cls._detect_encoding(path)
        f = codecs.open(path, "r", encoding=enc)
        bom = CODECS_BOMS.get(enc)
        if bom:
            head = f.read(len(bom))
            if head != bom:
                f.seek(0)
        return f, enc

    @classmethod
    def _handle_error(cls, err, handling, index):
        if handling == cls.ERROR_RAISE:
            err.args = (index,) + err.args
            raise err
        if handling == cls.ERROR_LOG:
            sys.stderr.write(f"PySRT-{type(err).__name__}(line {index}):\n")
            sys.stderr.write(err.args[0])
            sys.stderr.write("\n")

In [ ]:
# | eval: false
subs = SubRipFile.open("../samples/example.srt")
len(subs)

9

In [ ]:
# | eval: false
subs[0].start, subs[0].end, subs[0].text

(SubRipTime(0, 0, 0, 0),
 SubRipTime(0, 0, 2, 500),
 'Welcome to the Example Subtitle File!')

In [ ]:
# | eval: false
subs.shift(minutes=30)
subs[0].start, subs[0].end, subs[0].text

(SubRipTime(1, 0, 0, 0),
 SubRipTime(1, 0, 2, 500),
 'Welcome to the Example Subtitle File!')